# 1. 데이터 시각화 및 기본 통계량

## 1.1. 실습 개요

정형 데이터(Structured Data)를 분석할 때는 데이터를 단순히 불러오는 것뿐 아니라 **형태, 구조, 분포, 관계**를 면밀히 확인하는 과정이 필수적입니다.

이 노트북에서는 데이터의 기초 통계량 확인부터 시각화 기반 분석까지, EDA(탐색적 데이터 분석)의 핵심 절차를 학습합니다.

## 1.2. 학습 목표

1. 코랩 환경에서 한글 폰트 설정하기
2. 데이터 기본 정보 및 결측치 확인하기
3. 변수 유형별 분류 (명목형/순서형/이산형/연속형)
4. seaborn/matplotlib을 활용한 시각화

## 1.3. 사전 준비

- 이전 노트북에서 생성한 `highway_accident_full.csv` 파일 업로드

In [ ]:
# ============================================
# 2. 한글 폰트 설정 (코랩 전용)
# ============================================
# - 코랩에서는 기본 폰트가 한글을 지원하지 않아 별도 설정 필요
# - 나눔고딕 폰트를 설치하고 matplotlib 기본 폰트로 교체
# - 참조: https://conding-note.tistory.com/335

# --------------------------------------------
# 2.1. 나눔고딕 폰트 설치
# --------------------------------------------
!apt-get update -qq
!apt-get install -y fonts-nanum

import matplotlib as mpl
import shutil

# --------------------------------------------
# 2.2. matplotlib 기본 폰트 교체
# --------------------------------------------
# - matplotlib 설정 파일 경로에서 기본 폰트 파일 위치 확인
# - DejaVuSans.ttf를 나눔고딕으로 덮어쓰기
root = mpl.matplotlib_fname().replace("matplotlibrc", "")
target_font = root + "fonts/ttf/DejaVuSans.ttf"
nanum_font = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"

shutil.copyfile(nanum_font, target_font)
print("기본 폰트를 나눔고딕으로 교체 완료")

# --------------------------------------------
# 2.3. matplotlib 캐시 삭제 및 테스트
# --------------------------------------------
# - 캐시 삭제로 새 폰트 강제 적용
!rm -rf ~/.cache/matplotlib

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

# 한글 출력 테스트
plt.figure(figsize=(4, 3))
plt.title("한글 폰트 정상 출력 확인")
plt.xlabel("가로축")
plt.ylabel("세로축")
plt.plot([1, 2, 3], [1, 4, 2])
plt.show()

In [ ]:
# ============================================
# 3. 데이터 로드
# ============================================

import pandas as pd

# - 이전 노트북(1-1)에서 저장한 CSV 파일을 로드
# - 코랩에 파일을 업로드한 후 실행
path = "/content/highway_accident_full.csv"
df = pd.read_csv(path)

print("데이터 로드 완료")
df.head()

In [ ]:
# ============================================
# 4. 날짜/시간 데이터 전처리
# ============================================

# --------------------------------------------
# 4.1. 사고일자에서 연/월/일 추출
# --------------------------------------------
# - pd.to_datetime(): 문자열을 datetime 형식으로 변환
# - dt.year, dt.month, dt.day: datetime에서 연/월/일 추출
df["사고일자"] = pd.to_datetime(df["사고일자"])
df["사고년도"] = df["사고일자"].dt.year
df["사고월"] = df["사고일자"].dt.month
df["사고일"] = df["사고일자"].dt.day
df = df.drop(columns=["사고일자"])

# --------------------------------------------
# 4.2. 사고시각에서 시/분 추출
# --------------------------------------------
# - format="%H:%M": 시:분 형식 지정
# - dt.hour, dt.minute: 시/분 추출
df["사고시각"] = pd.to_datetime(df["사고시각"], format="%H:%M")
df["사고시"] = df["사고시각"].dt.hour
df["사고분"] = df["사고시각"].dt.minute
df = df.drop(columns=["사고시각"])

print("전처리 완료")
df.head()

In [ ]:
# ============================================
# 5. 데이터 기본 정보 확인
# ============================================

# --------------------------------------------
# 5.1. 데이터 크기 및 컬럼 확인
# --------------------------------------------
print("▶ 데이터 크기 (shape):", df.shape)
print("\n▶ 컬럼 목록:", df.columns.tolist())

# --------------------------------------------
# 5.2. 데이터 타입 및 메모리 정보
# --------------------------------------------
print("\n▶ 데이터 정보 (info)")
df.info()

# --------------------------------------------
# 5.3. 결측치 확인
# --------------------------------------------
# - isnull().sum(): 각 컬럼별 결측치 개수
# - isnull().mean(): 각 컬럼별 결측치 비율
print("\n▶ 결측치 개수")
print(df.isnull().sum())

print("\n▶ 결측치 비율")
print(df.isnull().mean().round(4))

# --------------------------------------------
# 5.4. 상위 5개 행 확인
# --------------------------------------------
print("\n▶ 상위 5개 행")
display(df.head(5))

In [ ]:
# ============================================
# 6. 변수 유형 분류: 수치형 vs 문자형
# ============================================

import numpy as np

# - select_dtypes(): 데이터 타입에 따라 컬럼 선택
# - np.number: 모든 수치형 타입 (int, float)
# - "object": 문자열 타입
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
object_cols = df.select_dtypes(include=["object"]).columns.tolist()

print("▶ 수치형 컬럼:", numeric_cols)
print("▶ 문자형 컬럼:", object_cols)

In [ ]:
# ============================================
# 7. 변수 유형 세분화: 명목형/순서형/이산형/연속형
# ============================================

# --------------------------------------------
# 7.1. 범주형 변수 분류
# --------------------------------------------
# - 순서형(ordinal): 순서가 있는 범주 (예: 등급, 학년)
# - 명목형(nominal): 순서가 없는 범주 (예: 노선명, 원인)
ordinal_candidates = []  # 이 데이터에는 순서형 변수가 없음
nominal_cols = [c for c in object_cols if c not in ordinal_candidates]

# --------------------------------------------
# 7.2. 수치형 변수 분류
# --------------------------------------------
# - 이산형(discrete): 정수로 표현되는 값 (예: 사망자수, 월)
# - 연속형(continuous): 실수로 표현되는 값 (예: 거리)
# - 고유값 개수가 threshold 미만이면 이산형으로 분류
threshold = 70
discrete_cols = []
continuous_cols = []

for c in numeric_cols:
    if df[c].nunique() < threshold:
        discrete_cols.append(c)
    else:
        continuous_cols.append(c)

print("▶ 명목형(범주형) 컬럼:", nominal_cols)
print("▶ 순서형(ordinal) 컬럼:", ordinal_candidates)
print("▶ 이산형(discrete) 컬럼:", discrete_cols)
print("▶ 연속형(continuous) 컬럼:", continuous_cols)

In [ ]:
# ============================================
# 8. 명목형(Nominal) 변수 시각화
# ============================================

# - countplot(): 범주별 빈도수를 막대그래프로 시각화
# - value_counts(): 각 값의 빈도수 계산

for col in nominal_cols:
    print(f"\n==================== [명목형] {col} ====================")
    
    vc = df[col].value_counts(dropna=False)
    
    plt.figure(figsize=(8, 4))
    sns.countplot(data=df, x=col, order=vc.index)
    plt.title(f"{col} 분포 (고유값 {len(vc)}개)")
    plt.xlabel(col)
    plt.ylabel("빈도")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# ============================================
# 9. 이산형(Discrete) 변수 시각화
# ============================================

# - barplot(): 이산형 값의 빈도를 막대그래프로 시각화
# - boxplot(): 데이터 분포와 이상치 확인
# - 박스플롯 해석: https://bommbom.tistory.com/entry

for col in discrete_cols:
    print(f"\n==================== [이산형] {col} ====================")
    
    vc = df[col].value_counts().sort_index()
    
    # 막대그래프
    plt.figure(figsize=(8, 4))
    sns.barplot(x=vc.index, y=vc.values, palette="Set2")
    plt.title(f"{col} 분포 (이산형)")
    plt.xlabel(col)
    plt.ylabel("빈도")
    plt.tight_layout()
    plt.show()
    
    # 박스플롯
    plt.figure(figsize=(4, 4))
    sns.boxplot(y=df[col], color="lightgreen")
    plt.title(f"{col} 박스플롯")
    plt.tight_layout()
    plt.show()

In [ ]:
# ============================================
# 10. 연속형(Continuous) 변수 시각화
# ============================================

# - histplot(): 연속형 값의 분포를 히스토그램으로 시각화
# - kde=True: 커널 밀도 추정 곡선 추가
# - describe(): 기초 통계량 (평균, 표준편차, 사분위수 등)

for col in continuous_cols:
    print(f"\n==================== [연속형] {col} ====================")
    
    desc = df[col].describe()
    print(desc)
    print(f"\n▶ 평균(mean): {desc['mean']:.3f}")
    print(f"▶ 표준편차(std): {desc['std']:.3f}")
    
    # 히스토그램 + KDE
    plt.figure(figsize=(6, 4))
    sns.histplot(df[col], kde=True, bins=20, color="skyblue")
    plt.title(f"{col} 히스토그램 + KDE")
    plt.xlabel(col)
    plt.ylabel("빈도")
    plt.tight_layout()
    plt.show()
    
    # 박스플롯
    plt.figure(figsize=(4, 4))
    sns.boxplot(y=df[col], color="orange")
    plt.title(f"{col} 박스플롯")
    plt.tight_layout()
    plt.show()

In [ ]:
# ============================================
# 11. 수치형 변수 간 상관관계 분석
# ============================================

# - corr(): 피어슨 상관계수 계산 (-1 ~ 1)
# - heatmap(): 상관계수를 색상으로 시각화
# - 피어슨 상관계수 해석: https://dd0za-1004.tistory.com/46

all_numeric = discrete_cols + continuous_cols

if len(all_numeric) >= 2:
    corr = df[all_numeric].corr()
    print("▶ 상관계수 행렬:")
    print(corr.round(2))
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f", 
                linewidths=0.5, square=True)
    plt.title("수치형 변수 상관관계 히트맵")
    plt.tight_layout()
    plt.show()
else:
    print("상관관계를 계산할 수치형 변수가 2개 미만입니다.")

In [ ]:
# ============================================
# 12. 변수 간 관계 시각화: 페어플롯
# ============================================

# - pairplot(): 모든 수치형 변수 쌍의 산점도와 히스토그램을 한 번에 시각화
# - 대각선: 각 변수의 히스토그램
# - 비대각선: 두 변수 간 산점도

numeric_df = df.select_dtypes(include=[np.number])
print("사용되는 수치형 변수:", numeric_df.columns.tolist())

sns.pairplot(numeric_df)
plt.suptitle("수치형 변수 페어플롯", y=1.02)
plt.tight_layout()
plt.show()

# 13. 다음 단계 안내

## 13.1. 이번 노트북에서 배운 내용

1. 코랩에서 한글 폰트 설정 방법
2. 데이터 기본 정보 확인 (shape, info, 결측치)
3. 변수 유형 분류 (명목형/순서형/이산형/연속형)
4. 변수 유형별 시각화 방법
5. 상관관계 분석 (히트맵, 페어플롯)

## 13.2. 다음 노트북에서 학습할 내용

다음 노트북 `2_1_EDA_및_시각화_심화.ipynb`에서는:

1. 파생변수 생성 (Feature Engineering)
2. 다차원 EDA (노선별, 시간대별, 요일별 분석)
3. 피벗 테이블 활용
4. 데이터 기반 의사결정 방향 도출

## 13.3. 추가 참고 자료

- [박스플롯 해석 방법](https://bommbom.tistory.com/entry)
- [피어슨 상관계수 이해](https://dd0za-1004.tistory.com/46)
- [도로교통공단 교통사고 상세데이터](https://www.data.go.kr/data/15070295/fileData.do)

In [ ]:
# ============================================
# 14. 전처리된 데이터 저장
# ============================================

# - 다음 노트북에서 사용할 수 있도록 전처리된 데이터 저장
csv_path = "/content/highway_accident_preprocessed.csv"
df.to_csv(csv_path, index=False, encoding="utf-8-sig")

print(f"저장 완료: {csv_path}")
print(f"저장된 데이터 크기: {df.shape}")